In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip3 install datasets -q
!pip3 install gensim -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.3.1+cu121 requires nvidia-cublas-cu12==12.1.3.1; platform_system == "Linux" and platform_machine == "x86_64", which is not installed.
torch 2.3.1+cu121 requires nvidia-cuda-cupti-cu12==12.1.105; platform_system == "Linux" and platform_machine == "x86_64", which is not installed.
torch 2.3.1+cu121 requires nvidia-cuda-nvrtc-cu12==12.1

In [3]:
!pip3 install nltk -q
!pip3 install regex -q
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [4]:
from datasets import load_dataset
ds = load_dataset("google-research-datasets/paws", "labeled_final")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/49401 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/8000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/8000 [00:00<?, ? examples/s]

In [5]:
ds

DatasetDict({
    train: Dataset({
        features: ['id', 'sentence1', 'sentence2', 'label'],
        num_rows: 49401
    })
    test: Dataset({
        features: ['id', 'sentence1', 'sentence2', 'label'],
        num_rows: 8000
    })
    validation: Dataset({
        features: ['id', 'sentence1', 'sentence2', 'label'],
        num_rows: 8000
    })
})

In [6]:
ds['train']['sentence1'][0]

'In Paris , in October 1560 , he secretly met the English ambassador , Nicolas Throckmorton , asking him for a passport to return to England through Scotland .'

In [7]:
ds['train']['sentence2'][0]

'In October 1560 , he secretly met with the English ambassador , Nicolas Throckmorton , in Paris , and asked him for a passport to return to Scotland through England .'

In [8]:
train_sentence1 = ds['train']['sentence1']
train_sentence2 = ds['train']['sentence2']
train_labels = ds['train']['label']

val_sentence1 = ds['validation']['sentence1']
val_sentence2 = ds['validation']['sentence2']
val_labels = ds['validation']['label']

test_sentence1 = ds['test']['sentence1']
test_sentence2 = ds['test']['sentence2']
test_labels = ds['test']['label']

train_sentence1 = train_sentence1[:15000]
train_sentence2 = train_sentence2[:15000]
train_labels = train_labels[:15000]

val_sentence1 = val_sentence1[:4000]
val_sentence2 = val_sentence2[:4000]
val_labels = val_labels[:4000]

print(len(train_sentence1), len(train_sentence2))
print(len(val_sentence1), len(val_sentence2))
print(len(test_sentence1), len(test_sentence2))

15000 15000
4000 4000
8000 8000


In [9]:
avg_len = 0
for i in range(len(train_sentence1)):
    avg_len += len(train_sentence1[i].split()) + len(train_sentence2[i].split())

for i in range(len(test_sentence1)):
    avg_len += len(test_sentence1[i].split()) + len(test_sentence2[i].split())

avg_len//= (len(train_sentence1) + len(train_sentence2) + len(test_sentence1) + len(test_sentence2))

avg_len

21

In [10]:
import re
from nltk.corpus import stopwords

stop_words = set(stopwords.words('english'))

max_len = 0
for i in range(len(train_sentence1)):
    max_len = max(max_len, len(train_sentence1[i].split()), len(train_sentence2[i].split()))

for i in range(len(test_sentence1)):
    max_len = max(max_len, len(test_sentence1[i].split()), len(test_sentence2[i].split()))

def preprocess(text, max_len):
    text = re.sub(r'[^\w\s]', '', text).lower()
    text = [word for word in text.split() if word not in stop_words]

    if len(text) > max_len:
        text = text[:max_len]

    if len(text) < max_len:
        text.extend(['EXTRA_TOKEN'] * (max_len - len(text)))


    return ' '.join(text)




In [11]:
max_len

37

In [12]:
preprocess(train_sentence1[0],max_len)

'paris october 1560 secretly met english ambassador nicolas throckmorton asking passport return england scotland EXTRA_TOKEN EXTRA_TOKEN EXTRA_TOKEN EXTRA_TOKEN EXTRA_TOKEN EXTRA_TOKEN EXTRA_TOKEN EXTRA_TOKEN EXTRA_TOKEN EXTRA_TOKEN EXTRA_TOKEN EXTRA_TOKEN EXTRA_TOKEN EXTRA_TOKEN EXTRA_TOKEN EXTRA_TOKEN EXTRA_TOKEN EXTRA_TOKEN EXTRA_TOKEN EXTRA_TOKEN EXTRA_TOKEN EXTRA_TOKEN EXTRA_TOKEN'

In [13]:
preprocess(train_sentence1[0],25)

'paris october 1560 secretly met english ambassador nicolas throckmorton asking passport return england scotland EXTRA_TOKEN EXTRA_TOKEN EXTRA_TOKEN EXTRA_TOKEN EXTRA_TOKEN EXTRA_TOKEN EXTRA_TOKEN EXTRA_TOKEN EXTRA_TOKEN EXTRA_TOKEN EXTRA_TOKEN'

In [14]:
preprocess(train_sentence2[0],avg_len)

'october 1560 secretly met english ambassador nicolas throckmorton paris asked passport return scotland england EXTRA_TOKEN EXTRA_TOKEN EXTRA_TOKEN EXTRA_TOKEN EXTRA_TOKEN EXTRA_TOKEN EXTRA_TOKEN'

In [15]:
train_s1 = [preprocess(text, 25) for text in train_sentence1]
train_s2 = [preprocess(text, 25) for text in train_sentence2]

val_s1 = [preprocess(text, 25) for text in val_sentence1]
val_s2 = [preprocess(text, 25) for text in val_sentence2]

test_s1 = [preprocess(text, 25) for text in test_sentence1]
test_s2 = [preprocess(text, 25) for text in test_sentence2]

print(len(train_s1), len(train_s2), len(train_labels))
print(len(val_s1), len(val_s2), len(val_labels))
print(len(test_s1), len(test_s2), len(test_labels))

15000 15000 15000
4000 4000 4000
8000 8000 8000


In [16]:
train_s1[0]

'paris october 1560 secretly met english ambassador nicolas throckmorton asking passport return england scotland EXTRA_TOKEN EXTRA_TOKEN EXTRA_TOKEN EXTRA_TOKEN EXTRA_TOKEN EXTRA_TOKEN EXTRA_TOKEN EXTRA_TOKEN EXTRA_TOKEN EXTRA_TOKEN EXTRA_TOKEN'

In [17]:
import gensim
fasttext_model_path = "/content/drive/MyDrive/FASTTEXT_embedding_model/wiki-news-300d-1M.vec"
embedding_model = gensim.models.KeyedVectors.load_word2vec_format(fasttext_model_path)

In [18]:
import numpy as np
def generate_embeddings(text, model):
    embeddings = []
    words = text.split()
    for word in words:
        if word == 'EXTRA_TOKEN' or word not in model:
            embeddings.append(np.zeros(model.vector_size))
        else:
            embeddings.append(model[word])

    return embeddings

train_s1_embeddings = [generate_embeddings(text, embedding_model) for text in train_s1]
train_s2_embeddings = [generate_embeddings(text, embedding_model) for text in train_s2]

val_s1_embeddings = [generate_embeddings(text, embedding_model) for text in val_s1]
val_s2_embeddings = [generate_embeddings(text, embedding_model) for text in val_s2]

test_s1_embeddings = [generate_embeddings(text, embedding_model) for text in test_s1]
test_s2_embeddings = [generate_embeddings(text, embedding_model) for text in test_s2]

In [19]:
embedding_dim = len(train_s1_embeddings[0][0])
max_len = 25
embedding_dim

300

In [20]:
train_s1_embeddings = np.array(train_s1_embeddings)
train_s2_embeddings = np.array(train_s2_embeddings)

val_s1_embeddings = np.array(val_s1_embeddings)
val_s2_embeddings = np.array(val_s2_embeddings)

test_s1_embeddings = np.array(test_s1_embeddings)
test_s2_embeddings = np.array(test_s2_embeddings)

train_labels = np.array(train_labels)
val_labels = np.array(val_labels)
test_labels = np.array(test_labels)

In [36]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, concatenate
from sklearn.metrics import accuracy_score

embedding_dim = len(train_s1_embeddings[0][0])
sequence_length = max_len


input_s1 = Input(shape=(sequence_length, embedding_dim))
input_s2 = Input(shape=(sequence_length, embedding_dim))

lstm = LSTM(128, return_sequences=False)

encoded_s1 = lstm(input_s1)
encoded_s2 = lstm(input_s2)

merged = concatenate([encoded_s1, encoded_s2])

dense = Dense(64, activation='relu')(merged)
output = Dense(1, activation='sigmoid')(dense)

model = Model(inputs=[input_s1, input_s2], outputs=output)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.fit([train_s1_embeddings, train_s2_embeddings],
          train_labels,
          validation_data=([val_s1_embeddings, val_s2_embeddings], val_labels),
          epochs=10,
          batch_size=4)

test_predictions = model.predict([test_s1_embeddings, test_s2_embeddings])
test_predictions = np.round(test_predictions).astype(int)

Epoch 1/10
3750/3750 ━━━━━━━━━━━━━━━━━━━━ 31s 8ms/step - accuracy: 0.5508 - loss: 0.6896 - val_accuracy: 0.5562 - val_loss: 0.6872
Epoch 2/10
3750/3750 ━━━━━━━━━━━━━━━━━━━━ 31s 8ms/step - accuracy: 0.5550 - loss: 0.6879 - val_accuracy: 0.5558 - val_loss: 0.6870
Epoch 3/10
3750/3750 ━━━━━━━━━━━━━━━━━━━━ 37s 7ms/step - accuracy: 0.5618 - loss: 0.6857 - val_accuracy: 0.5570 - val_loss: 0.6868
Epoch 4/10
3750/3750 ━━━━━━━━━━━━━━━━━━━━ 41s 7ms/step - accuracy: 0.5605 - loss: 0.6833 - val_accuracy: 0.5575 - val_loss: 0.6863
Epoch 5/10
3750/3750 ━━━━━━━━━━━━━━━━━━━━ 41s 7ms/step - accuracy: 0.5764 - loss: 0.6767 - val_accuracy: 0.5537 - val_loss: 0.6912
Epoch 6/10
3750/3750 ━━━━━━━━━━━━━━━━━━━━ 42s 8ms/step - accuracy: 0.5830 - loss: 0.6695 - val_accuracy: 0.5575 - val_loss: 0.6880
Epoch 7/10
3750/3750 ━━━━━━━━━━━━━━━━━━━━ 40s 7ms/step - accuracy: 0.5906 - loss: 0.6655 - val_accuracy: 0.5560 - val_loss: 0.6919
Epoch 8/10
3750/3750 ━━━━━━━━━━━━━━━━━━━━ 41s 7ms/step - accuracy: 0.6097 - loss: 0

In [35]:
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 25, 300)        │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_1             │ (None, 25, 300)        │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm (LSTM)               │ (None, 128)            │        219,648 │ input_layer[0][0],     │
│                           │                        │                │ input_layer_1[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate (Concatenate) │ (None, 256)            │              0 │ lstm[0][0], lstm[1][0] │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense (Dense)             │ (None, 64)             │         16,448 │ concatenate[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_1 (Dense)           │ (None, 1)              │             65 │ dense[0][0]            │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 708,485 (2.70 MB)

 Trainable params: 236,161 (922.50 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 472,324 (1.80 MB)

In [37]:
accuracy_score(test_predictions, test_labels)

0.62425

In [23]:
train_s1_embeddings.shape

(15000, 25, 300)

In [24]:
def flatten_embeddings(embeddings):
    num_samples = embeddings.shape[0]
    return embeddings.reshape(num_samples, -1)

train_s1_flat = flatten_embeddings(train_s1_embeddings[:5000])
train_s2_flat = flatten_embeddings(train_s2_embeddings[:5000])

val_s1_flat = flatten_embeddings(val_s1_embeddings[:1000])
val_s2_flat = flatten_embeddings(val_s2_embeddings[:1000])

test_s1_flat = flatten_embeddings(test_s1_embeddings[:1000])
test_s2_flat = flatten_embeddings(test_s2_embeddings[:1000])

In [25]:
train_ml_data = np.concatenate([train_s1_flat, train_s2_flat], axis=1)
val_ml_data = np.concatenate([val_s1_flat, val_s2_flat], axis=1)
test_ml_data = np.concatenate([test_s1_flat, test_s2_flat], axis=1)

In [29]:
from sklearn.decomposition import PCA

pca = PCA(n_components=100)
train_features_pca = pca.fit_transform(train_ml_data)
val_features_pca = pca.transform(val_ml_data)
test_features_pca = pca.transform(test_ml_data)

In [30]:
from sklearn.svm import LinearSVC

svm_model = LinearSVC(dual=False, max_iter=5000, C=0.1)
svm_model.fit(train_features_pca, train_labels[:5000])

val_predictions = svm_model.predict(val_features_pca)
val_accuracy = accuracy_score(val_labels[:1000], val_predictions)
print(f"Validation Accuracy: {val_accuracy:.4f}")

test_predictions = svm_model.predict(test_features_pca)
test_accuracy = accuracy_score(test_labels[:1000], test_predictions)
print(f"Test Accuracy: {test_accuracy:.4f}")


Validation Accuracy: 0.5420
Test Accuracy: 0.5350


In [31]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(train_features_pca, train_labels[:5000])

val_predictions = rf_model.predict(val_features_pca)
val_accuracy = accuracy_score(val_labels[:1000], val_predictions)
print(f"Validation Accuracy with Random Forest: {val_accuracy:.4f}")

test_predictions = rf_model.predict(test_features_pca)
test_accuracy = accuracy_score(test_labels[:1000], test_predictions)
print(f"Test Accuracy with Random Forest: {test_accuracy:.4f}")

Validation Accuracy with Random Forest: 0.5310
Test Accuracy with Random Forest: 0.5620


In [32]:
from sklearn.linear_model import LogisticRegression

lr_model = LogisticRegression(max_iter=1000, random_state=42)
lr_model.fit(train_features_pca, train_labels[:5000])

val_predictions = lr_model.predict(val_features_pca)
val_accuracy = accuracy_score(val_labels[:1000], val_predictions)
print(f"Validation Accuracy with Logistic Regression: {val_accuracy:.4f}")

test_predictions = lr_model.predict(test_features_pca)
test_accuracy = accuracy_score(test_labels[:1000], test_predictions)
print(f"Test Accuracy with Logistic Regression: {test_accuracy:.4f}")

Validation Accuracy with Logistic Regression: 0.5400
Test Accuracy with Logistic Regression: 0.5370


In [34]:
from sklearn.neighbors import KNeighborsClassifier

knn_model = KNeighborsClassifier(n_neighbors=5)
knn_model.fit(train_features_pca, train_labels[:5000])

val_predictions = knn_model.predict(val_features_pca)
val_accuracy = accuracy_score(val_labels[:1000], val_predictions)
print(f"Validation Accuracy with KNN: {val_accuracy:.4f}")

test_predictions = knn_model.predict(test_features_pca)
test_accuracy = accuracy_score(test_labels[:1000], test_predictions)
print(f"Test Accuracy with KNN: {test_accuracy:.4f}")

Validation Accuracy with KNN: 0.5090
Test Accuracy with KNN: 0.5290
